In [ ]:
import random
import os
import json
import cv2
import numpy as np
import math
from scipy import ndimage
from skimage import exposure

In [ ]:
# Step 1: Fetch the JSON file
with open('C://Users//Kashan//Desktop//cat_dog//cat_dog_annotations.json', 'r') as f:
    vgg_annotations = json.load(f)

# Initialize the COCO annotation dictionary
coco_annotations = {
    'info': {},
    'licenses': [],
    'categories': [],
    'images': [],
    'annotations': []
}

In [27]:
def generate_augmented_image_name(image_id, angle, brightness_factor, resized_scale):
    image_name = str(image_id).split(".")[0]
    return f"{image_name}_{angle}_{brightness_factor:.2f}_{resized_scale:.2f}.jpeg"

for image_id, image_data in vgg_annotations['_via_img_metadata'].items():
    # Extract the image name without extension
    image_filename = image_data['filename']
    image_name = os.path.splitext(image_filename)[0]

    # Load the image
    image_path = os.path.join('C:\\Users\\Kashan\\Desktop\\cat_dog', image_filename)
    image = cv2.imread(image_path)

    # Get the annotations for the image
    annotations = image_data['regions']

    # Step 7: Repeat steps 3 to 6 for all angles
    angles = range(0, 360, 5)  # 0, 5, 10, 15, ..., 350, 355
    for angle in angles:
        # Step 3: Resize the image and adjust bounding boxes
        resized_scale = random.uniform(0.6, 1.01)
        resize_width = int(image.shape[1] * resized_scale)
        resize_height = int(image.shape[0] * resized_scale)
        resized_image = cv2.resize(image, (resize_width, resize_height))

        resized_annotations = []
        for annotation in annotations:
            shape_attributes = annotation['shape_attributes']
            x = shape_attributes['x'] * resized_scale
            y = shape_attributes['y'] * resized_scale
            width = shape_attributes['width'] * resized_scale
            height = shape_attributes['height'] * resized_scale

            resized_annotation = {
                'id': len(resized_annotations) + 1,
                'image_id': int(''.join(filter(str.isdigit, image_id))),
                'category_id': 0,  # Update with the correct category ID
                'bbox': [x, y, width, height],
                'area': width * height,
                'iscrowd': 0,
                'segmentation': []
            }
            resized_annotations.append(resized_annotation)

        # Step 4: Add random brightness
        brightness_factor = random.uniform(0.5, 1.5)
        brighter_image = exposure.adjust_gamma(resized_image, brightness_factor)

        # Step 5: Rotate the brighter image on an angle n and adjust bounding boxes
        rotated_image = ndimage.rotate(brighter_image, angle, reshape=False)

        rotated_annotations = []
        for annotation in resized_annotations:
            bbox = annotation['bbox']
            x, y, width, height = bbox

            rad_angle = math.radians(angle)
            rotated_x = math.cos(rad_angle) * x - math.sin(rad_angle) * y
            rotated_y = math.sin(rad_angle) * x + math.cos(rad_angle) * y

            rotated_bbox = [rotated_x, rotated_y, width, height]

            rotated_annotation = {
                'id': len(rotated_annotations) + 1,
                'image_id': int(''.join(filter(str.isdigit, image_id))),
                'category_id': 0,  # Update with the correct category ID
                'bbox': rotated_bbox,
                'area': width * height,
                'iscrowd': 0,
                'segmentation': []
            }
            rotated_annotations.append(rotated_annotation)

        # Step 6: Save the resulting image on disk
        # Generate the filename based on the image_id, angle, brightness_factor, and resize_scale_percentage
        output_filename = generate_augmented_image_name(image_id, angle, brightness_factor, resized_scale)

        # Save the rotated_image with the generated filename
        output_path = os.path.join('C:\\Users\\Kashan\\Desktop\\augmented_images', output_filename)
        cv2.imwrite(output_path, rotated_image)

        # Step 9: Save the new annotations in COCO format
        coco_annotations['images'].append({
            'id': int(''.join(filter(str.isdigit, image_id))),
            'file_name': output_filename,
            'width': rotated_image.shape[1],
            'height': rotated_image.shape[0],
            'license': 0
        })

        coco_annotations['annotations'].extend(rotated_annotations)

# Save the COCO annotations to a JSON file
with open('C:\\Users\\Kashan\\Desktop\\augmented_images\\coco_annotations.json', 'w') as f:
    json.dump(coco_annotations, f, indent=4)